In [ ]:
import pandas as pd
import panel as pn
import plotly.express as px
pn.extension('plotly')
# Carregar dados
df_info = pd.read_csv('csv/common_player_info.csv')
df_player = pd.read_csv('csv/player.csv')
# Preprocessamento de altura e peso
def parse_height(h):
    if isinstance(h, str) and '-' in h:
        ft, inch = h.split('-')
        return round(int(ft) * 0.3048 + int(inch) * 0.0254, 2)
    return None
df_info['altura_m'] = df_info['height'].apply(parse_height)
df_info['peso_kg'] = pd.to_numeric(df_info['weight'], errors='coerce')
# Filtros
times = sorted(df_info['team_name'].dropna().unique().tolist())
posicoes = sorted(df_info['position'].dropna().unique().tolist())
time_selector = pn.widgets.Select(name='Time', options=['Todos'] + times, value='Todos', width=300)
pos_selector = pn.widgets.Select(name='Posição', options=['Todos'] + posicoes, value='Todos', width=300)
def filtrar():
    df = df_info.copy()
    if time_selector.value != 'Todos':
        df = df[df['team_name'] == time_selector.value]
    if pos_selector.value != 'Todos':
        df = df[df['position'] == pos_selector.value]
    return df
# Gráfico de distribuição de altura
def grafico_altura(time, pos):
    df = filtrar()
    fig = px.histogram(df, x='altura_m', nbins=20, labels={'altura_m': 'Altura (m)'}, title='Distribuição de Jogadores por Altura')
    fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(title='Contagem de Jogadores')
    return fig
# Gráfico de distribuição de peso
def grafico_peso(time, pos):
    df = filtrar()
    fig = px.histogram(df, x='peso_kg', nbins=20, labels={'peso_kg': 'Peso (libras)'}, title='Distribuição de Jogadores por Peso')
    fig.update_layout(height=350, plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    fig.update_yaxes(title='Contagem de Jogadores')
    return fig
# Gráfico de dispersão Altura vs Peso por posição
def grafico_dispersao(time, pos):
    df = filtrar()
    # Merge para pegar nome completo
    df = df.merge(df_player[['id', 'full_name']], left_on='person_id', right_on='id', how='left')
    fig = px.scatter(
        df,
        x='altura_m',
        y='peso_kg',
        color='position',
        hover_data={'full_name': True, 'position': True, 'altura_m': True, 'peso_kg': True},
        labels={'altura_m': 'Altura (m)', 'peso_kg': 'Peso (libras)', 'position': 'Posição'},
        title='Altura vs. Peso por Posição',
        height=400
    )
    fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', paper_bgcolor='rgba(0,0,0,0)')
    return fig
# Painel final
dashboard = pn.Column(
    pn.Row(
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Distribuição de Jogadores por Altura</h2>'),
            pn.bind(grafico_altura, time_selector, pos_selector),
        ),
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Distribuição de Jogadores por Peso</h2>'),
            pn.bind(grafico_peso, time_selector, pos_selector),
        ),
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Filtros</h2>'),
            pn.pane.Markdown('<b>Time</b>'),
            time_selector,
            pn.pane.Markdown('<b>Posição</b>'),
            pos_selector,
        ),
    ),
    pn.Row(
        pn.Column(
            pn.pane.Markdown('<h2 style="color:#bdf;">Altura vs. Peso por Posição</h2>'),
            pn.bind(grafico_dispersao, time_selector, pos_selector),
        ),
    ),
)
dashboard